<a href="https://colab.research.google.com/github/hamidzangiabadi/LSTM-HHO-IoMT-Colab/blob/main/Thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random
import math
import time

In [9]:

def fitness_function(x):
    return np.sum(x**2)


def Levy(dim):
    beta = 1.5
    sigma = ( math.gamma(1+beta) * math.sin(math.pi*beta/2) /
             ( math.gamma((1+beta)/2) * beta * 2**((beta-1)/2) ) )**(1/beta)
    u = 0.01 * np.random.randn(dim) * sigma
    v = np.random.randn(dim)
    zz = np.power(np.abs(v), (1/beta))
    step = u / zz
    return step

def initialize_population(pop_size, dim, lb, ub):
    return lb + np.random.rand(pop_size, dim) * (ub - lb)



def HHO(objf, lb, ub, dim, SearchAgents_no, max_iter):

    # Rabbit position (best solution so far)
    Rabbit_Location = np.zeros(dim)
    Rabbit_Energy = float("inf")

    # Hawks positions
    X = initialize_population(SearchAgents_no, dim, lb, ub)

    # Convergence curve
    convergence_curve = np.zeros(max_iter)

    print(f"HHO is now tackling '{objf.__name__}'")
    timer_start = time.time()
    for t in range(max_iter):
        # --- ارزیابی قبل از حرکت ---
        for i in range(SearchAgents_no):
            X[i, :] = np.clip(X[i, :], lb, ub)
            fitness = objf(X[i, :])
            if fitness < Rabbit_Energy:
                Rabbit_Energy = fitness
                Rabbit_Location = X[i, :].copy()

        E1 = 2 * (1 - t / max_iter)

        # --- حرکت Hawks ---
        for i in range(SearchAgents_no):
            E0 = 2 * random.random() - 1
            Escaping_Energy = E1 * E0

            if abs(Escaping_Energy) >= 1:
                # Exploration Eq.1
                q = random.random()
                if q >= 0.5:
                    r1 = random.random()
                    r2 = random.random()
                    rand_idx = math.floor(SearchAgents_no * random.random())
                    X_rand = X[rand_idx, :]
                    X[i, :] = X_rand - r1 * abs(X_rand - 2 * r2 * X[i, :])
                else:
                    r3 = random.random()
                    r4 = random.random()
                    X_mean = np.mean(X, axis=0)
                    random_point = lb + r4 * (ub - lb)
                    X[i, :] = (Rabbit_Location - X_mean) - r3 * (random_point)
            else:
                # Exploitation Eq.4-11
                r = random.random()
                Jump_Strength = 2 * (1 - random.random())

                if r >= 0.5 and abs(Escaping_Energy) >= 0.5:
                    DX = Rabbit_Location - X[i, :]
                    X[i, :] = DX - Escaping_Energy * abs(Jump_Strength * Rabbit_Location - X[i, :])

                if r >= 0.5 and abs(Escaping_Energy) < 0.5:
                    X[i, :] = Rabbit_Location - Escaping_Energy * abs(Rabbit_Location - X[i, :])

                if r < 0.5 and abs(Escaping_Energy) >= 0.5:
                    X1 = Rabbit_Location - Escaping_Energy * abs(Jump_Strength * Rabbit_Location - X[i, :])
                    if objf(X1) < objf(X[i, :]):
                        X[i, :] = X1.copy()
                    else:
                        X2 = X1 + np.multiply(np.random.randn(dim), Levy(dim))
                        if objf(X2) < objf(X[i, :]):
                            X[i, :] = X2.copy()

                if r < 0.5 and abs(Escaping_Energy) < 0.5:
                    X1 = Rabbit_Location - Escaping_Energy * abs(Jump_Strength * Rabbit_Location - np.mean(X, axis=0))
                    if objf(X1) < objf(X[i, :]):
                        X[i, :] = X1.copy()
                    else:
                        X2 = X1 + np.multiply(np.random.randn(dim), Levy(dim))
                        if objf(X2) < objf(X[i, :]):
                            X[i, :] = X2.copy()

        # --- ارزیابی بعد از حرکت ---
        for i in range(SearchAgents_no):
            X[i, :] = np.clip(X[i, :], lb, ub)
            fitness = objf(X[i, :])
            if fitness < Rabbit_Energy:
                Rabbit_Energy = fitness
                Rabbit_Location = X[i, :].copy()

        convergence_curve[t] = Rabbit_Energy
        print(f"Iteration {t+1}/{max_iter} | Best fitness: {Rabbit_Energy:.6f}")

    timer_end = time.time()
    execution_time = timer_end - timer_start

    return {
        "best_fitness": Rabbit_Energy,
        "best_position": Rabbit_Location,
        "convergence_curve": convergence_curve,
        "execution_time": execution_time
    }


# ------------------------
# Run test
# ------------------------
if __name__ == "__main__":
    result = HHO(fitness_function, lb=-10, ub=10, dim=5,
                 SearchAgents_no=10, max_iter=100)

    print("\nBest Solution:", result["best_position"])
    print("Best Fitness:", result["best_fitness"])


HHO is now tackling 'fitness_function'
Iteration 1/100 | Best fitness: 91.203249
Iteration 2/100 | Best fitness: 7.917641
Iteration 3/100 | Best fitness: 2.088779
Iteration 4/100 | Best fitness: 2.088779
Iteration 5/100 | Best fitness: 0.724425
Iteration 6/100 | Best fitness: 0.042736
Iteration 7/100 | Best fitness: 0.042736
Iteration 8/100 | Best fitness: 0.042691
Iteration 9/100 | Best fitness: 0.032041
Iteration 10/100 | Best fitness: 0.032041
Iteration 11/100 | Best fitness: 0.027630
Iteration 12/100 | Best fitness: 0.027630
Iteration 13/100 | Best fitness: 0.027630
Iteration 14/100 | Best fitness: 0.023276
Iteration 15/100 | Best fitness: 0.005485
Iteration 16/100 | Best fitness: 0.002875
Iteration 17/100 | Best fitness: 0.002875
Iteration 18/100 | Best fitness: 0.002875
Iteration 19/100 | Best fitness: 0.001374
Iteration 20/100 | Best fitness: 0.001374
Iteration 21/100 | Best fitness: 0.001374
Iteration 22/100 | Best fitness: 0.001374
Iteration 23/100 | Best fitness: 0.001374
Ite